In [ ]:
import pandas as pd
import os

def ausentismo(file_path):
    # Normalize and remove surrounding quotes from the file path
    file_path = os.path.normpath(file_path.strip('"').strip("'"))

    # Separate columns
    df = pd.read_excel(file_path)
    df = df['EmployeeID|"EmployeeName"|"Organization"|"Department"|"Date"|"ShiftType"|"OfficialHours"|"ActualHours"|"ExceptionType"|"Reason"|"Details"|"Comments"'].str.split('|',
            expand=True)
    df.columns = ['EmployeeID', 'EmployeeName', 'Organization', 'Department', 'Date', 'ShiftType', 'OfficialHours', 'ActualHours', 'ExceptionType', 'Reason',
                'Details', 'Comments']
    df = df[df['EmployeeID'] != 'EmployeeID']

    # Filter data
    df['ShiftType'] = df['ShiftType'].str.replace('"', '', regex=False)
    df['EmployeeName'] = df['EmployeeName'].str.replace('"', '', regex=False)
    df['Date'] = df['Date'].str.replace('"', '', regex=False)
    df['Details'] = df['Details'].str.replace('"', '', regex=False)
    df['Comments'] = df['Comments'].str.replace('"', '', regex=False)
    df['ExceptionType'] = df['ExceptionType'].str.replace('"', '', regex=False)
    df = df[df['ShiftType'] == 'Shift']
    excep_terms = ['Absence', 'Justification']
    df = df[df['ExceptionType'].isin(excep_terms)]

    # Drop unnecessary columns
    df = df.drop(columns=['Organization', 'Department', 'OfficialHours', 'ActualHours', 'Reason', 'ShiftType'])

    # Add columns from another Excel file (with fake column names)
    df2 = pd.read_excel(r"C:\Path\To\DF_FAKE.xlsx")  # Use fake dataset
    df2.rename(columns={'Fake Employee ID': 'EmployeeID'}, inplace=True)
    df2['EmployeeID'] = df2['EmployeeID'].astype(str)
    df_agency = df2[['EmployeeID', 'Subarea']]
    df_zone = df2[['EmployeeID', 'Unit']]
    df_position = df2[['EmployeeID', 'Position']]
    df_go = df2[['EmployeeID', 'Division']]
    df['EmployeeID'] = df['EmployeeID'].astype(str)
    df = df.merge(df_agency, on='EmployeeID', how='left')
    df = df.merge(df_zone, on='EmployeeID', how='left')
    df = df.merge(df_position, on='EmployeeID', how='left')
    df = df.merge(df_go, on='EmployeeID', how='left')

    # Homologate position title with fake categories
    df['Position'] = df['Position'].str.upper()

    categories = {
        'WAREHOUSE ASSISTANT': ['WAREHOUSE ASST', 'WH ASSISTANT'],
        'SALES REP': ['SALES REPRESENTATIVE', 'BD REP'],
        'OPERATIONS MGR': ['OPERATIONS MANAGER', 'OPS MGR'],
        'SUPERVISOR': ['SUPERVISOR', 'SUPV'],
    }

    for category, terms in categories.items():
        mask = df['Position'].str.startswith(tuple(terms), na=False)
        df.loc[mask, 'Position'] = category

    # Select operative positions
    operative_positions = ['WAREHOUSE ASSISTANT', 'SALES REP']
    df = df[df['Position'].isin(operative_positions)]

    # Add column AGENCY (fake data)
    df['Agency'] = df['Subarea'].str[:-2]

    # Select agencies with time block (use fake names)
    agencies = ['AGENCY A', 'AGENCY B', 'AGENCY C']  # Anonymized agency names
    df = df[df['Agency'].str.startswith(tuple(agencies))]

    df = df[df['Agency'] != 'EXCLUDED AGENCY']

    # Save the updated DataFrame back to the Excel file
    df.to_excel(file_path, index=False)

file_path = input("Enter the path to the Excel file: ")

ausentismo(file_path)
